<a href="https://colab.research.google.com/github/Ali-Hasan-Khan28/Natural-Language-Processing/blob/main/Lab11b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Bidirectional

In [2]:
data = pd.DataFrame({
    'word': ["John", "Smith", "works", "at", "Google", "in", "New", "York"],
    'pos_tag': ["NNP", "NNP", "VBZ", "IN", "NNP", "IN", "NNP", "NNP"],
    'label': ["B-PER", "I-PER", "O", "O", "B-ORG", "O", "B-LOC", "I-LOC"]
})

In [3]:
words = data['word'].tolist()
labels = data['label'].tolist()
num_words = len(set(words))
num_labels = len(set(labels))
word2idx = {word: idx + 1 for idx, word in enumerate(set(words))}
label2idx = {label: idx for idx, label in enumerate(set(labels))}
X = [[word2idx[word] for word in words]]
y = [[label2idx[label] for label in labels]]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X * 100, y * 100, test_size=0.2, random_state=42)

In [5]:
def create_lstm_model(max_len, num_words, num_labels):
    input_layer = Input(shape=(max_len,))
    embedding_layer = Embedding(input_dim=num_words, output_dim=50, input_length=max_len)(input_layer)
    lstm_layer = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(embedding_layer)
    output_layer = TimeDistributed(Dense(num_labels, activation='softmax'))(lstm_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
def train_lstm_model(X_train, y_train, X_test, y_test, num_words, num_labels):
    max_len = max(len(seq) for seq in X_train + X_test)
    X_train_pad = pad_sequences(X_train, maxlen=max_len, padding='post', value=0)  # Add 'value=0' for padding token
    y_train_pad = pad_sequences(y_train, maxlen=max_len, padding='post', value=0)  # Add 'value=0' for padding token
    X_test_pad = pad_sequences(X_test, maxlen=max_len, padding='post', value=0)    # Add 'value=0' for padding token
    y_test_pad = pad_sequences(y_test, maxlen=max_len, padding='post', value=0)    # Add 'value=0' for padding token

    model = create_lstm_model(max_len, num_words, num_labels)
    model.fit(X_train_pad, to_categorical(y_train_pad, num_classes=num_labels), validation_data=(X_test_pad, to_categorical(y_test_pad, num_classes=num_labels)), batch_size=32, epochs=10)
    return model

In [11]:
# Step 5: Inference
# Evaluate the model's performance using standard NER metrics such as precision, recall, and F1-score
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=-1)
    y_test_labels = np.array(y_test)
    # Flatten the sequences and ignore padding
    y_pred_labels_flat = [label for pred_seq, true_seq in zip(y_pred_labels, y_test_labels) for label in pred_seq[:len(true_seq)]]
    y_test_labels_flat = [label for seq in y_test_labels for label in seq]
    print(classification_report(y_test_labels_flat, y_pred_labels_flat))

In [15]:
# Train and evaluate the LSTM model
model = train_lstm_model(X_train, y_train, X_test, y_test, num_words + 1, num_labels + 1)  # Increment num_words and num_labels to account for padding token

# Calculate max_len
max_len = max(len(seq) for seq in X_train + X_test)

# Retrieve padded test data from the train_lstm_model function
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding='post', value=0)    # Add 'value=0' for padding token
y_test_pad = pad_sequences(y_test, maxlen=max_len, padding='post', value=0)    # Add 'value=0' for padding token

# Evaluate the model
evaluate_model(model, X_test_pad, y_test_pad)


Epoch 1/10
3/3 [==============================] - 9s 313ms/step - loss: 1.9380 - accuracy: 0.3594 - val_loss: 1.9160 - val_accuracy: 0.3750
Epoch 2/10
3/3 [==============================] - 0s 53ms/step - loss: 1.9076 - accuracy: 0.3750 - val_loss: 1.8829 - val_accuracy: 0.3750
Epoch 3/10
3/3 [==============================] - 0s 52ms/step - loss: 1.8726 - accuracy: 0.3750 - val_loss: 1.8418 - val_accuracy: 0.3750
Epoch 4/10
3/3 [==============================] - 0s 45ms/step - loss: 1.8285 - accuracy: 0.3750 - val_loss: 1.7878 - val_accuracy: 0.3750
Epoch 5/10
3/3 [==============================] - 0s 46ms/step - loss: 1.7698 - accuracy: 0.3750 - val_loss: 1.7146 - val_accuracy: 0.3750
Epoch 6/10
3/3 [==============================] - 0s 56ms/step - loss: 1.6906 - accuracy: 0.3750 - val_loss: 1.6168 - val_accuracy: 0.3750
Epoch 7/10
3/3 [==============================] - 0s 47ms/step - loss: 1.5857 - accuracy: 0.3750 - val_loss: 1.4964 - val_accuracy: 0.3750
Epoch 8/10
3/3 [==========

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
